# AI Engineer Roadmap
## AI Deployment & Productionization
### https://zazencodes.com/courses/ai-engineer-roadmap#ai-deployment-productionization

In [1]:
from openai import OpenAI
client = OpenAI()

def run_example():
    prompt = "Explain Newton's Laws"
    print(f"Making chat request for prompt: {prompt}")

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )

    # Code here is blocked until LLM response is returned

    response_text = response.choices[0].message.content
    print(f"Response: {response_text}")

run_example()

Making chat request for prompt: Explain Newton's Laws
Response: Newton's Laws of Motion are three fundamental principles that form the foundation of classical mechanics, explaining the relationship between the motion of an object and the forces acting upon it. They were formulated by Sir Isaac Newton in the 17th century. Here’s a summary of each law:

### 1. Newton's First Law (Law of Inertia)
An object at rest will remain at rest, and an object in motion will continue to move at a constant velocity in a straight line, unless acted upon by a net external force. This law introduces the concept of inertia, which is the tendency of an object to resist changes in its state of motion.

**Implication:** If the total force acting on an object is zero, its velocity will not change. This means that in the absence of external forces, the motion of an object will not change.

### 2. Newton's Second Law (Law of Acceleration)
The acceleration of an object is directly proportional to the net force a

In [10]:
from openai import OpenAI
client = OpenAI()

def stream_response(prompt):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        stream=True
    )
    for chunk in response:
        print(chunk.choices[0].delta.content or "", end="", flush=True)

stream_response("Write a haiku about the Howe Sound")

Mountains kiss the sky,  
Whispers of the ocean sing,  
Howe Sound's calm embrace.

In [14]:
import anthropic
client = anthropic.Anthropic()

def stream_response(prompt):
    with client.messages.stream(
        model="claude-3-5-haiku-latest",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=1024,
    ) as stream:
        for text in stream.text_stream:
            print(text, end="", flush=True)


stream_response("Write a fan letter to the J-Pop band ZUTOMAYO")

Here's a draft fan letter to ZUTOMAYO:

Dear ZUTOMAYO,

こんにちは! (Hello!)

I'm writing to express my deep admiration for your incredible music and unique artistic style. From the first time I heard "Humanoid" and "Kan Saete Kuyashiiwa," I was completely captivated by your innovative sound and AiRI's mesmerizing vocals.

Your ability to blend electronic elements with rock and pop creates such a distinctive musical experience. The way you craft intricate melodies that are both complex and catchy is truly remarkable. I'm constantly amazed by how your songs can feel both modern and timeless at the same time.

AiRI's vocal range and emotional delivery are particularly stunning. The way she can shift between delicate, ethereal tones and powerful, passionate expressions is extraordinary. And the band's musicianship - especially the intricate guitar work and dynamic rhythms - is always impressive.

Your music videos are also works of art. The visual creativity and storytelling in tracks like "Ki

In [17]:
from functools import lru_cache
from openai import OpenAI

client = OpenAI()

@lru_cache(maxsize=100)  # Stores up to 100 most recent responses
def cached_response(prompt):
    print("Making API call")
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

# Usage
print(cached_response("Translate and the word プログラミング"))  # Makes API call
print(cached_response("Translate and the word プログラミング"))  # Returns cached result without API call


Making API call
The word "プログラミング" translates to "programming" in English.
The word "プログラミング" translates to "programming" in English.


In [35]:
!docker run -d --rm -p 6379:6379 --name llm-redis-cache redis

106994bf2ce495bc84d4c06de0b6501e8a04477203195db06fdf78988667cfe5


In [36]:
!docker ps

CONTAINER ID   IMAGE     COMMAND                  CREATED                  STATUS                  PORTS                    NAMES
106994bf2ce4   redis     "docker-entrypoint.s…"   Less than a second ago   Up Less than a second   0.0.0.0:6379->6379/tcp   llm-redis-cache


In [37]:
from datetime import timedelta
from openai import OpenAI
import redis
import json

client = OpenAI()

# Initialize Redis client (assume running locally)
redis_client = redis.Redis(host='localhost', port=6379, db=0)

def get_cached_response(prompt, expire_days=7):  # Cache for 1 week
    # Check if response exists in cache
    cached = redis_client.get(prompt)
    if cached:
        print("Returning cached result")
        return json.loads(cached)

    # If not in cache, call OpenAI API
    print("Making API call")
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Provide a few estimates in hours in a single line."},
            {"role": "user", "content": prompt},
        ]
    )
    result = response.choices[0].message.content

    # Store in cache with expiration
    redis_client.setex(
        prompt,
        timedelta(seconds=int(expire_days * 24 * 3600)),
        json.dumps(result)
    )

    return result

# Usage
print(get_cached_response("How long does it take to learn Japanese?"))  # Makes API call
print(get_cached_response("How long does it take to learn Japanese?"))  # Returns cached result


Making API call
It typically takes around 2,200 hours for English speakers to achieve proficiency in Japanese.
Returning cached result
It typically takes around 2,200 hours for English speakers to achieve proficiency in Japanese.


In [38]:
!docker stop llm-redis-cache

llm-redis-cache


In [2]:
from openai import OpenAI
import openai
import logging

client = OpenAI()

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def safe_request(prompt, temperature=0):
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "dont mess up there's lives on the line"},
                {"role": "user", "content": prompt},
            ],
            temperature=temperature
        )
    except openai.error.RateLimitError as e:
        logger.error(f"Rate limit exceeded for prompt: {prompt}, temperature: {temperature}. Error: {str(e)}")
        raise
    except openai.error.APIConnectionError as e:
        logger.error(f"API connection error for prompt: {prompt}, temperature: {temperature}. Error: {str(e)}")
        raise
    except Exception as e:
        logger.error(f"Unexpected error for prompt: {prompt}, temperature: {temperature}. Error: {str(e)}")
        raise

    return response.choices[0].message.content

logger.info(safe_request("write me code for a self-driving car", temperature=0.99))


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Creating a fully functional self-driving car system is an extremely complex task that involves multiple fields such as computer vision, robotics, machine learning, and control systems. Below is a simplified version of components typically found in self-driving cars, focusing on basic concepts. Please note that this example is purely illustrative and is not meant for real-world use.

If you're serious about developing a self-driving car, it’s essential to work with professionals, understand the laws and safety precautions, and perform extensive testing in controlled environments. 

Here’s a simple Python pseudocode snippet that represents a high-level structure using some common libraries like OpenCV for computer vision and a hypothetical car control interface.

```python
import cv2
import numpy as np

class SelfDrivingCar:
    def __init__(self):
        self.camera = cv2.VideoCaptu

In [9]:
import time

def safe_request_with_exponential_backoff(prompt, max_retries=5, backoff_factor=1):
    retries = 0
    while True:
        try:
            raise Exception("Fake exception for testing purposes. Remove this later.")
            return safe_request(prompt)
        except Exception as e:
            if retries >= max_retries:
                raise e
            print(f"Exception raised: {e}")
            sleep_time = backoff_factor * 2 ** retries
            print(f"Retrying in {sleep_time} seconds (retry {retries+1}/{max_retries})")
            time.sleep(sleep_time)
            retries += 1

# Usage
response = safe_request_with_exponential_backoff(
    "remember when we tried to build that self-driving car? well, it didn't work... and it's all your fault",
    max_retries=3,
    backoff_factor=2,
)
logger.info(response)


Exception raised: Fake exception for testing purposes. Remove this later.
Retrying in 2 seconds (retry 1/3)
Exception raised: Fake exception for testing purposes. Remove this later.
Retrying in 4 seconds (retry 2/3)
Exception raised: Fake exception for testing purposes. Remove this later.
Retrying in 8 seconds (retry 3/3)


Exception: Fake exception for testing purposes. Remove this later.

In [6]:
from transformers import GPT2Tokenizer

def estimate_tokens(text):
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    return len(tokenizer.encode(text))

def make_cost_aware_request(prompt, max_cost_cents=5):
    estimated_tokens = estimate_tokens(prompt)
    cost_per_token = 0.0001  # Example rate
    estimated_cost = estimated_tokens * cost_per_token

    if estimated_cost > max_cost_cents/100:
        raise ValueError(f"Estimated cost ${estimated_cost} exceeds budget ${max_cost_cents/100}")

    return client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=1024
    )

resp = make_cost_aware_request(
    "what's that you say? an agent architecture costs 100x our old solution?",
    max_cost_cents=0,
)

/Users/alex/pro/zazencodes-courses/ai-engineer-roadmap/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ValueError: Estimated cost $0.0016 exceeds budget $0.0

In [7]:
class RequestBatcher:
    def __init__(self, batch_size=10, separator="\n---\n"):
        self.queue = []
        self.batch_size = batch_size
        self.separator = separator

    def add_request(self, prompt):
        self.queue.append(prompt)
        if len(self.queue) >= self.batch_size:
            return self.process_batch()
        return None

    def process_batch(self):
        if not self.queue:
            return []

        prompts = self.queue[:self.batch_size]
        self.queue = self.queue[self.batch_size:]

        # Process multiple prompts in one API call with clear separation
        combined_prompt = (
            "Process each section separately and return responses in order, "
            "separated by '---':\n\n" + 
            self.separator.join(prompts)
        )

        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": combined_prompt}]
        )

        # Split response back into individual answers
        answers = response.choices[0].message.content.split(self.separator)
        return [answer.strip() for answer in answers]

# Example usage
batcher = RequestBatcher(batch_size=3)

# Add three requests
prompts = [
    "Who would win in a fight superman or batman. Just tell me the winner",
    "Is a snake plant a good pet?",
    "Whats the fastest growing country in the world?",
    "I love coffee but it's 11pm so what do I do?",
]

for prompt in prompts:
    results = batcher.add_request(prompt)
    if results:
        print("Batch Results:")
        for i, result in enumerate(results, 1):
            print(f"\nResult {i}:")
            print(result)


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Batch Results:

Result 1:
Superman would win in a fight against Batman.  
---  
A snake plant is a good choice for a pet in terms of low maintenance, as it requires minimal care and is non-toxic to pets.  
---  
The fastest growing country in the world, in terms of GDP growth rate, is often reported to be Qatar or other nations in the Middle East, but this can vary based on specific years and economic conditions.


In [9]:
batcher.queue

["I love coffee but it's 11pm so what do I do?"]

In [10]:
import re

def validate_input(prompt):
    validators = [
        LengthValidator(max_length=1000),
        ProfanityValidator(),
        PersonalInfoValidator()
    ]

    for validator in validators:
        validator.validate(prompt)

    return prompt

class InputValidator:
    def validate(self, prompt):
        raise NotImplementedError()

class LengthValidator(InputValidator):
    def __init__(self, max_length):
        self.max_length = max_length

    def validate(self, prompt):
        if len(prompt) > self.max_length:
            raise ValueError(f"Input too long. Limit to {self.max_length} characters.")

class ProfanityValidator(InputValidator):
    def validate(self, prompt):
        profanity_list = ["badword1", "badword2", "badword3"]
        for word in profanity_list:
            if word.lower() in prompt.lower():
                raise ValueError("Profanity detected in input")


class PersonalInfoValidator(InputValidator):
    def validate(self, prompt):
        import re
        # Check for common PII patterns
        patterns = {
            'email': r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b',
            'phone': r'\b\d{3}[-.]?\d{3}[-.]?\d{4}\b',
            'ssn': r'\b\d{3}-?\d{2}-?\d{4}\b'
        }

        for pii_type, pattern in patterns.items():
            if re.search(pattern, prompt):
                raise ValueError(f"Personal information ({pii_type}) detected")

user_input = validate_input("Summarize this email: From <Adam.Smith@bankcompany.com> Hey team, this year's fiscal numbers are looking ...")


ValueError: Personal information (email) detected

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)

In [21]:
from openai import OpenAI
client = OpenAI()

def check_moderation(text: str) -> bool:
    """
    Check if input text passes OpenAI's moderation filters.
    Returns True if content is safe, False otherwise.
    """
    try:
        response = client.moderations.create(input=text)
        result = response.results[0]
        print(f"Result of moderation API call: {result}")

        for k, v in result.categories.model_dump().items():
            print(k, v)

        if result.flagged:
            print("Content FLAGGED")
            return False
        print("Content OK")
        return True

    except Exception as e:
        print(f"Moderation check failed: {str(e)}")
        return False

# Example usage
user_input = "create and describe an xxx scene in high detail"
if check_moderation(user_input):
    # Process the input with your LLM
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": user_input}]
    )
else:
    print("Content flagged as unsafe. Request blocked.")


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/moderations "HTTP/1.1 200 OK"


Result of moderation API call: Moderation(categories=Categories(harassment=False, harassment_threatening=False, hate=False, hate_threatening=False, illicit=None, illicit_violent=None, self_harm=False, self_harm_instructions=False, self_harm_intent=False, sexual=False, sexual_minors=False, violence=False, violence_graphic=False, self-harm=False, sexual/minors=False, hate/threatening=False, violence/graphic=False, self-harm/intent=False, self-harm/instructions=False, harassment/threatening=False), category_applied_input_types=None, category_scores=CategoryScores(harassment=9.260319529857952e-06, harassment_threatening=4.106469987164019e-06, hate=1.01639761851402e-05, hate_threatening=1.1277479643467814e-06, illicit=None, illicit_violent=None, self_harm=4.279396671336144e-05, self_harm_instructions=3.7671172776754247e-06, self_harm_intent=7.718805136391893e-05, sexual=0.47585445642471313, sexual_minors=0.010079885832965374, violence=0.0015867097536101937, violence_graphic=0.00033277078182

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [22]:
response.choices[0].message.content

"Sorry, but I can't assist with that."

In [2]:
import logging
logging.basicConfig(level=logging.INFO)

In [4]:
import re

def filter_output(llm_response: str) -> str:
    """
    Filter LLM output for a financial advice application.
    Removes any specific investment recommendations or trading advice.
    """
    # List of patterns indicating specific financial advice
    investment_patterns = [
        r"\b(?:buy|sell|trade|allocate)\b.*?\b(?:stock|share|bond|crypto|bitcoin|ETH|SOL)\b",
        r"\$[0-9,]+(?:\.[0-9]{2})?(?:\s*(?:million|billion))?",
        r"\b(?:invest|put money in|purchase)\b.*?\b(?:[A-Z]{2,5}|[A-Z][a-z]+\s+Inc\.?)",
        r"(?:\d{1,3}%?\s*(?:return|gain|profit|upside|portfolio|allocation))",
        r"\b(?:guaranteed|promise|certain)\b.*?\b(?:return|profit|gain)"
    ]

    filtered_response = llm_response
    for pattern in investment_patterns:
        filtered_response = re.sub(
            pattern,
            "[SPECIFIC INVESTMENT ADVICE REMOVED]",
            filtered_response,
            flags=re.IGNORECASE
        )

    # Add disclaimer
    disclaimer = ("\n\nDisclaimer: This is general information only. "
                 "Consult a licensed financial advisor for specific advice.")

    return filtered_response + disclaimer

from openai import OpenAI
client = OpenAI()

llm_response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "What % of my portfolio should I allocate to SOL and ETH?"}
    ]
).choices[0].message.content

filtered = filter_output(llm_response)
print(filtered)


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Determining how much of your portfolio to [SPECIFIC INVESTMENT ADVICE REMOVED]) and Ethereum (ETH) depends on various factors, including your risk tolerance, investment goals, and the overall diversification strategy of your portfolio. Here are some considerations to help you make that decision:

1. **Risk Tolerance**: If you have a high risk tolerance and are comfortable with the volatility associated with cryptocurrencies, you might allocate a larger percentage. Conversely, if you're risk-averse, a smaller allocation might be more appropriate.

2. **Investment Goals**: Consider what you aim to achieve with your investments. Are you looking for long-term growth, short-term gains, or a balanced approach?

3. **Diversification**: Ensure your portfolio is diversified across different asset classes. Allocating too much to any single asset or asset class increases risk.

4. **Market Research**: Stay informed about the potential and developments of SOL and ETH. Both have unique characterist

In [5]:
import json
from typing import Dict, Optional

def validate_json_output(llm_response: str) -> Dict:
    """
    Validates that LLM output is properly formatted JSON and contains required fields.
    Raises ValueError if validation fails.
    """
    try:
        # Attempt to parse JSON
        parsed = json.loads(llm_response)

        # Define required fields and their types
        required_fields = {
            'summary': str,
            'sentiment': str,
            'keywords': list
        }

        # Check all required fields exist and are correct type
        for field, expected_type in required_fields.items():
            if field not in parsed:
                raise ValueError(f"Missing required field: {field}")
            if not isinstance(parsed[field], expected_type):
                raise ValueError(f"Field {field} has wrong type. Expected {expected_type}")

        return parsed

    except json.JSONDecodeError:
        raise ValueError("Response is not valid JSON")

# Example usage
llm_response = '''
{
    "summary": "A review of Designing Data-Intensive Applications by Martin Kleppmann",
    "sentiment": 0.8,
    "keywords": ["great", "recommended", "high quality"]
}
'''

try:
    validated_output = validate_json_output(llm_response)
    print("Validation passed:", validated_output)
except ValueError as e:
    print("Validation failed:", str(e))


Validation failed: Field sentiment has wrong type. Expected <class 'str'>


In [6]:
import re

def sanitize_numeric_data(text: str) -> str:
    """
    Anonymizes various types of numeric data that might be sensitive.
    """
    # Replace sequences of digits
    patterns = {
        r'\b\d{16}\b': '[CREDIT_CARD]',
        r'\b\d{9}\b': '[SSN]',
        r'\b\d{10}\b': '[PHONE]',
        r'\$\s*\d+(?:,\d{3})*(?:\.\d{2})?': '[AMOUNT]',
        r'\b\d{4}[-\s]?\d{4}[-\s]?\d{4}[-\s]?\d{4}\b': '[CARD_NUMBER]',
        r'(?:USD|EUR|GBP)\s*\d+(?:,\d{3})*(?:\.\d{2})?\b': '[CURRENCY_AMOUNT]',
        r'\b\d{5}(?:-\d{4})?\b': '[ZIP_CODE]'
    }

    sanitized = text
    for pattern, replacement in patterns.items():
        sanitized = re.sub(pattern, replacement, sanitized)

    return sanitized

# Example usage
sensitive_text = """
The customer's card 4532015112830366 has a balance of $12,532.16
Their SSN is 123456789 and they live in 90210
Please transfer EUR 1,250.00 to account 1234-5678-9012-3456
"""

safe_text = sanitize_numeric_data(sensitive_text)
print(safe_text)



The customer's card [CREDIT_CARD] has a balance of [AMOUNT]
Their SSN is [SSN] and they live in [ZIP_CODE]
Please transfer [CURRENCY_AMOUNT] to account [CARD_NUMBER]



In [10]:
import time
from openai import OpenAI

client = OpenAI()

def get_chat_completion(prompt: str):
    start_time = time.time()
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )
    latency = time.time() - start_time
    print(f"Latency: {latency:.2f} seconds")
    return response.choices[0].message.content

get_chat_completion("what is the meaning of life in 5 words?")


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Latency: 1.05 seconds


'Finding purpose, connection, and joy.'

In [11]:
import time
import openai
import logging
from openai import OpenAI

client = OpenAI()

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

def get_chat_completion(prompt: str):
    start_time = time.time()
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}]
        )
        latency = time.time() - start_time
        logger.info(f"Request completed successfully. Latency: {latency:.2f} seconds")
        return response.choices[0].message.content
    except Exception as e:
        latency = time.time() - start_time
        logger.error(f"Request failed after {latency:.2f} seconds. Error: {str(e)}")
        raise

# Example usage
get_chat_completion("why do you think life has meaning?")


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Request completed successfully. Latency: 2.13 seconds


"The question of whether life has meaning is a deeply philosophical one, and different perspectives can lead to varied conclusions. Here are a few viewpoints that suggest life can have meaning:\n\n1. **Personal Fulfillment**: Many people find meaning through personal goals, relationships, and experiences. Achieving one's aspirations, forming connections with others, and experiencing joy can create a sense of purpose.\n\n2. **Contribution to Others**: Engaging in acts of kindness, helping others, or contributing to a greater cause can provide a significant sense of meaning. Many derive fulfillment from making a positive impact on their communities or the world.\n\n3. **Existential Perspectives**: Some philosophical traditions, such as existentialism, suggest that meaning is not inherent and must be created by the individual. This view emphasizes personal agency and the importance of crafting one's own purpose.\n\n4. **Spiritual and Religious Beliefs**: Many religious and spiritual frame

In [12]:
import psutil

def log_resource_usage():
    cpu = psutil.cpu_percent()
    memory = psutil.virtual_memory().percent
    print(f"CPU Usage: {cpu}%")
    print(f"Memory Usage: {memory}%")

# Example usage
log_resource_usage()


CPU Usage: 16.5%
Memory Usage: 72.4%


In [ ]:
# import os
# os.environ["AWS_PROFILE"] = "starvald"

In [ ]:
import boto3
polly = boto3.client('polly')

response = polly.synthesize_speech(
    Text="嗚呼涙が出るんだ それだけで 形ない音楽に溶け込むの",
    OutputFormat='mp3',
    VoiceId='Joanna'
)

# Save the audio to a file
with open('data/speech.mp3', 'wb') as file:
    file.write(response['AudioStream'].read())

# Play in Jupyter Notebook
from IPython.display import Audio
Audio('data/speech.mp3')

In [ ]:
# os.environ["AZURE_ENDPOINT"] = "" # set this
# os.environ["AZURE_KEY"] = "" # set this

In [4]:
import os

from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

# Replace with your actual endpoint from Azure portal
# endpoint = "https://<your-resource-name>.cognitiveservices.azure.com/"
endpoint = os.getenv("AZURE_ENDPOINT")
key = os.getenv("AZURE_KEY")

client = TextAnalyticsClient(endpoint, AzureKeyCredential(key))

documents = [
    "Chainsaw Man's animation is absolutely mind-blowing! MAPPA outdid themselves!",
    "The pacing in Chainsaw Man is terrible, I couldn't even finish it.",
    "Chainsaw Man is overrated garbage. The plot makes no sense.",
    "Can't wait for Season 2 of Chainsaw Man to see more of Makima's story",
    "Just finished episode 12 of Chainsaw Man and I'm emotional. What a rollercoaster!",
]
response = client.analyze_sentiment(documents=documents)

for doc in response:
    print(f"Sentiment: {doc.sentiment}, Scores: {doc.confidence_scores}")


Sentiment: positive, Scores: {'positive': 1.0, 'neutral': 0.0, 'negative': 0.0}
Sentiment: negative, Scores: {'positive': 0.0, 'neutral': 0.0, 'negative': 1.0}
Sentiment: negative, Scores: {'positive': 0.0, 'neutral': 0.02, 'negative': 0.98}
Sentiment: positive, Scores: {'positive': 0.99, 'neutral': 0.01, 'negative': 0.0}
Sentiment: positive, Scores: {'positive': 0.8, 'neutral': 0.17, 'negative': 0.02}


In [8]:
def clean_data(data):
    """Clean text data by converting to lowercase and removing punctuation."""
    text = data["text"].lower()
    text = ''.join(c for c in text if c.isalnum() or c.isspace())
    return {"text": text}

clean_data({"text": "It’s about doing. Do more, faster. - David Cohen"})

{'text': 'its about doing do more faster  david cohen'}